Raster CLI and API sidebyside

In [1]:
import os
os.chdir(r"C:\Users\deleo\Downloads\buffer")
os.getcwd()

'C:\\Users\\deleo\\Downloads\\buffer'

In [12]:
from osgeo import gdal, ogr
import numpy as np
import  pandas as pd

#input files
raster_path = 'non_ahp_co_reproj.tif'
shapefile_path = 'bufer_250m.shp'

ds = gdal.Open(raster_path)
band = ds.GetRasterBand(1)
gt = ds.GetGeoTransform()
nodata = band.GetNoDataValue()

# Pixel Area
pixel_width = gt[1]
pixel_height = abs(gt[5])
pixel_area = pixel_width * pixel_height

#Opn shp
driver = ogr.GetDriverByName("ESRI Shapefile")
shp = driver.Open(shapefile_path)
layer = shp.GetLayer()

# Mask
mask_ds = gdal.GetDriverByName("MEM").Create(
    '', ds.RasterXSize, ds.RasterYSize, 1, gdal.GDT_Byte)
mask_ds.SetGeoTransform(gt)
mask_ds.SetProjection(ds.GetProjection())

# Rasterize buffer
gdal.RasterizeLayer(mask_ds, [1], layer, burn_values=[1])

# Read data
mask = mask_ds.GetRasterBand(1).ReadAsArray()
data = band.ReadAsArray()

# Apply mask
masked = np.where(mask == 1, data, np.nan)

# Get unique classes and pixel counts
unique, counts = np.unique(masked[~np.isnan(masked)], return_counts=True)

# Compute area
areas_m2 = counts * pixel_area
areas_ha = areas_m2 / 10000

# Put into table
df = pd.DataFrame({
    "class": unique.astype(int),
    "area_m2": areas_m2,
    "area_ha": areas_ha
})

print(df)
df.to_csv("non_ahp.csv", index=False)

                 class       area_m2     area_ha
0 -9223372036854775808  1.254094e+05   12.540941
1                   11  4.181822e+06  418.182208
2                   12  3.365239e+06  336.523874
3                   13  1.782688e+06  178.268832
4                   21  5.578779e+04    5.578779
5                   22  9.355284e+05   93.552835
6                   23  1.341622e+06  134.162213
7                   31  5.817962e+02    0.058180
8                   32  2.592225e+04    2.592225
9                   33  8.067575e+04    8.067575


C:\Users\deleo\AppData\Local\Temp\ipykernel_3912\1280776801.py:49: RuntimeWarning: invalid value encountered in cast
  "class": unique.astype(int),


In [ ]:
!ogrinfo C:\Users\deleo\Downloads\buffer\bufer_250m.shp -al -so

### Clipping Raster in the buffer


In [21]:
#files
raster_path = 'ahp_co_reproj.tif'
shapefile_path = 'bufer_250m.shp'
output_path = "ahp_250m.tif"

try: 
  shp = ogr.Open(shapefile_path)
  layer= shp.GetLayer()
  print("Clipping with", layer.GetName())
  
  gdal.Warp(
    output_path,
    raster_path,
    cutlineDSName=shapefile_path,
    cropToCutline="true"
  )
  print("Clipping successful " + output_path)
  
except AttributeError:
  print("No file exist, skipping gdal.Warp")


Clipping with bufer_250m
Clipping successful ahp_250m.tif
